In [20]:
#!/usr/bin/env python3
import os, platform, subprocess, sys

In [21]:
"""
Finding and importing provided libraries regardless of system and platform type
"""
## find and import libraries relative to this file's location (regardless of script or notebook)
## platfrom specific naming already handled within the lib directory's __init__ file
if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket
from lib.rpcauth  import rpcauth
from lib.sign     import sign_tx

In [22]:
'''
Step 0.0: setup our RPC socket like we did in the previous example
        If you have not done the previous example, now would be the time.
        Please copy in the variables you used (located conveniently at the end of the previous example)
'''

nodeIP = "127.0.0.1"
nodePort = "18444"
username = "user"
password = "password"
wallet = "DEV-TEST"

rpc = RpcSocket({ 'wallet': wallet,
                  'username': username,
                  'password': password,
                  'url' : nodeIP,
                  'port': nodePort})
assert rpc.check()

In [23]:
'''
Step 0.1: make sure you have cash. If you are broke, see previous example on how to get funds
'''
if float(rpc.call("getbalance")) == 0:
    raise Exception("get funds OR check if funds are still in a pending (ie. unconfirmed) state")
else:
    print("You have a balance of:", rpc.call("getbalance"), "BTC")

You have a balance of: 0.01091089 BTC


In [24]:
## First, we will lookup an existing utxo,
## and use that to fund our transaction.
## Get a utxo for Alice.
alice_utxo = rpc.get_utxo(0)
print(alice_utxo)

{'address': 'tb1qgrrt46ke9tk5l9xynd6vzh09q2z06qrwyqe8dl', 'priv_key': '3b912b3ddb272d85624514587c971ded7c0641f27c4502659cfe5a10605c03d2', 'pub_key': '03040036ef5dae681b75e236bbf951328bec5aa24d4112e77ade2aa25f2173a409', 'pubkey_hash': '40c6baead92aed4f94c49b74c15de50284fd006e', 'redeem_script': '1976a91440c6baead92aed4f94c49b74c15de50284fd006e88ac', 'txid': '8f37019947a8fe21f160ca104024425383472f361bd18f68ca761fab0de3a256', 'vout': 0, 'value': 6000}


In [29]:
## Get a change address for Alice.
alice_change_txout = rpc.get_recv(fmt='base58')
alice_pubkey_hash  = decode_address(alice_change_txout['address'])

print(alice_pubkey_hash)

## Get a payment address for Bob.
bob_payment_txout = rpc.get_recv(fmt='base58')
bob_pubkey_hash   = decode_address(bob_payment_txout['address'])

print(bob_pubkey_hash)

f5e02b6df07e25ca6b476a9dfadcb906d786696c
be65b05a2f53aa25ba01abf7fc59b58284542c4f


In [31]:
## Calculate our output amounts.
fee = 1000
bob_recv_value = alice_utxo['value'] // 2
alice_change_value = alice_utxo['value'] // 2 - fee

In [33]:
## The spending transaction.
atob_tx = {
    'version': 1,
    'vin': [{
        # We are unlocking the utxo from Alice.
        'txid': alice_utxo['txid'],
        'vout': alice_utxo['vout'],
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [
        {
            'value': bob_recv_value,
            'script_pubkey': ['OP_DUP', 'OP_HASH160', bob_pubkey_hash, 'OP_EQUALVERIFY', 'OP_CHECKSIG']
        },
        {
            'value': alice_change_value,
            'script_pubkey': ['OP_DUP', 'OP_HASH160', alice_pubkey_hash, 'OP_EQUALVERIFY', 'OP_CHECKSIG']
        }
    ],
    'locktime': 0
}


In [34]:
## Serialize the transaction and calculate the TXID.
atob_hex  = encode_tx(atob_tx)
atob_txid = hash256(bytes.fromhex(atob_hex))[::-1].hex()

In [35]:

## The redeem script is a basic Pay-to-Pubkey-Hash template.
redeem_script = f"76a914{alice_utxo['pubkey_hash']}88ac"

In [36]:

## We are signing Alice's UTXO using BIP143 standard.
alice_signature = sign_tx(
    atob_tx,                # The transaction.
    0,                      # The input being signed.
    alice_utxo['value'],    # The value of the utxo being spent.
    redeem_script,          # The redeem script to unlock the utxo.
    alice_utxo['priv_key']  # The private key to the utxo pubkey hash.
)

In [37]:

## Include the arguments needed to unlock the redeem script.
atob_tx['vin'][0]['witness'] = [ alice_signature, alice_utxo['pub_key'] ]

In [39]:

print(f'''
## Pay-to-Pubkey-Hash Example ##
-- Transaction Id --
{atob_txid}
-- Alice UTXO --
     Txid : {alice_utxo['txid']}
     Vout : {alice_utxo['vout']}
    Value : {alice_utxo['value']}
     Hash : {alice_utxo['pubkey_hash']}
-- Sending to Bob --
  Address : {bob_payment_txout['address']}
    Coins : {bob_recv_value}
-- Change --
  Address : {alice_change_txout['address']}
      Fee : {fee}
    Coins : {alice_change_value}
-- Hex --
{encode_tx(atob_tx)}
''')


## Pay-to-Pubkey-Hash Example ##
-- Transaction Id --
84da086f5650500ee2d7b02d340800d116aa6b026a8f9d3d171dd2222f29b2bb
-- Alice UTXO --
     Txid : 8f37019947a8fe21f160ca104024425383472f361bd18f68ca761fab0de3a256
     Vout : 0
    Value : 6000
     Hash : 40c6baead92aed4f94c49b74c15de50284fd006e
-- Sending to Bob --
  Address : mxsgVy9E4dZ77DiimKfnjRsk3WKDR4vRx1
    Coins : 3000
-- Change --
  Address : n3w2PdeB2Qc5VVS7hYrYxKaVS3GcZy4RYP
      Fee : 1000
    Coins : 2000
-- Hex --
0100000000010156a2e30dab1f76ca688fd11b362f47835342244010ca60f121fea8479901378f0000000000ffffffff02b80b0000000000001976a914be65b05a2f53aa25ba01abf7fc59b58284542c4f88acd0070000000000001976a914f5e02b6df07e25ca6b476a9dfadcb906d786696c88ac02483045022100a0ad8694d29a24c66b4a2096c25d99595607a38e50f17a814769466b3ccbf54102205089325fb476579f316f3ddae1a1ea6815e4d567b9ee5cdf2a6128df611a23de012103040036ef5dae681b75e236bbf951328bec5aa24d4112e77ade2aa25f2173a40900000000



In [38]:
rpc.transact(encode_tx(atob_tx))


'84da086f5650500ee2d7b02d340800d116aa6b026a8f9d3d171dd2222f29b2bb'